### tutorial on running inference on custom inputs

In [1]:
import pandas as pd
import numpy as np
import os

In [5]:
os.chdir('..')

### First find the pdb ids of the prots of intrest

Here we use pdbs from the sabdab test set used in AntiDIF / AntiFold/ ABMPNN

In [46]:
test_ids = pd.read_csv('data/raw_data/test_pdb_ids', index_col=0)
len(test_ids)

388

now to create the custom csv file we need the dir path that are pdb files will be in
pdb_file_dir

In [ ]:

def find_inps_csv(pdb_ids, dir, all_chains=True):
    resut = {}
    resut['pdb_paths'] = [dir + iden + '.pdb' for iden in pdb_ids]
    if all_chains:
        resut['chains'] = ['all'] * len(pdb_ids)
    else: 
        raise NotImplementedError('only all chains support so far')
    return pd.DataFrame(resut)

pdb_file_dir = '/Users/nik/Documents/tf_ox/code/inv_folding/data/sabdab/' #dir pdbs will be in

sab_test_inps = find_inps_csv(test_ids['0'], pdb_file_dir)
#sab_test_inps.to_csv()


Then simply save the df and update the path in config.yaml for custom_pdb_input to your custom csv.

# Model inference 

And
### Saving data as a pkl

Processing the data from a pdb can be time consuming if you want to re-run inference multiple times using the same data, here we save the data as a pickle file so that it can be quickly loaded in during inference. 

once saved set load_pkl to fpath of pkl dataset to load.

In [17]:
os.chdir('pi-rldif-ft')

In [73]:
from model.mod_pifold import InverseFoldingDiffusionPiFoldModel
from data.dataset import RLDIFDataset
from utils.utils import load_config
from torch.utils.data import DataLoader
import pickle as pkl
import torch

In [27]:
args = load_config('./configs/config.yaml')
master_config = load_config('./configs/master_config.yaml')

args.data.docker = False
args.data.rldif = args.rldif
args.data.protein_mpnn = args.protein_mpnn

args.data.custom_pdb_input

'data/raw_data/example.csv'

In [22]:
#test (have to change config.yaml for this)
custom_ds = RLDIFDataset(args.data) 

100%|██████████| 1/1 [00:00<00:00, 66.24it/s]

Entered 1 samples into Redis


In [25]:
with open('data/raw_data/example_ds.pkl', 'wb') as f:
    pkl.dump(custom_ds, f)

In [30]:
args.data.load_pkl = 'data/raw_data/example_ds.pkl'

In [33]:
if args.data.load_pkl:
    with open(args.data.load_pkl, 'rb') as f:
        dataset = pkl.load(f) 

In [58]:
## load model for inf
device = 'cpu'

In [ ]:
#load model and weights 

args.pifold_model.free_positions = args.free_positions
model = InverseFoldingDiffusionPiFoldModel(args.pifold_model).to(device)
m_path = args.m_name
state_dict = torch.load(m_path, map_location=device)['state_dict'] #n.b added map_location 
new_state_dict = {}
for k, v in state_dict.items():
    new_state_dict[k.replace('model.', '')] = v
model.load_state_dict(new_state_dict)

collate_function = model.collate_fn
dataloader = DataLoader(
                dataset,
                batch_size=1, 
                shuffle=False,
                collate_fn=collate_function,
            )

In [ ]:
# load model for inf need to also load weights.
for batch in dataloader:
    out = model.sample(batch.clone().to(device), closure=True)
    break
